In [1]:
import numpy as np
from itertools import product

In [2]:
# define number of letters to permute
N = 5
factorial = np.math.factorial
# number of elements in S_N
ORD = factorial(N)
# do you want to save your results in a file?
SAVE = True

In [3]:
# useful function for calculating cycle types from partitions
# e.g.: one_hot(3, 5) -> [0, 0, 0, 1, 0]
def one_hot(x, size):
    z = np.zeros(size, dtype="i4")
    z[x] = 1
    return z

# find all partitions for given integer
def get_partitions(n):
    if n == 0:
        yield np.array([], dtype="i4")
    if n < 0:
        return
    for p in get_partitions(n-1):
        yield np.concatenate((p, np.array([1])))
        l = len(p)
        if l == 1 or (l > 1 and p[-1] < p[-2]):
            yield p + one_hot(l-1, l)
            
# translate partition into cycle type
def get_cycle_type(p, l=None):
    if not l:
        l = max(p)
    # cycle type of one cycle
    kvec = lambda lam: one_hot(lam-1, l)
    # cycle types of all cycles in a list
    ks = np.vectorize(kvec, signature="()->(n)")(p)
    # return sum of all cycle types
    return np.sum(ks, axis=0)

# get all possible cycle types for elements in S_n
def get_all_cycle_types(n, l=None):
    for p in get_partitions(n):
        yield get_cycle_type(p, l)
                
# find all repartitions of a given partition
def get_repartitions(p, k):
    # cycle types of repartition should have the same length as input k
    l = len(k)
    # calculate all the cycle types of repartitions of each row
    reparts = [get_all_cycle_types(lam, l) for lam in p]
    # calculate cartesian product of the sets of repartitions of rows to 
    # get the repartitions of p
    reparts = np.array(list(product(*reparts)))
    # calculate the cycle types of the new repartitions
    reparts_ks = np.sum(reparts, axis=1)
    # accept only the repartitions with repart_k = k
    mask = np.product(reparts_ks == k, axis=1, dtype="bool")
    reparts = reparts[mask]
    # order along second axis of repartitions is reversed, but does not matter 
    # because of the product over j in (3.68) in the script
    return reparts

def psi(p, k):
    # calculate entries of Psi matrix
    res = np.array(0., dtype=np.float128)
    for r in get_repartitions(p, k):
        k_factorial = np.array([factorial(k_i) for k_i in k])
        r_factorial = np.array([[factorial(r_ij) for r_ij in r_j] for r_j in r])
        res += np.product(k_factorial)/np.product(r_factorial)
    return res

In [4]:
# build Psi matrix. The cycle types k need to be padded to length N
Psi = np.around(np.array([[psi(p, np.pad(k, (0, N-len(k)))) 
                          for k in get_all_cycle_types(N)] 
                          for p in get_partitions(N)])).astype(np.uint64)
print("Psi:\n", Psi, "\n")

# calculate order of conjugacy class labeled by cycle type k
def ord_C(k):
    # order of stabilizer
    ord_stab = np.product(np.array([(i+1)**k_i * factorial(k_i) 
                                    for (i, k_i) in enumerate(k)]))
    return ORD / ord_stab

# build Sigma matrix from (3.52) in the script
Sigma = np.diag(np.array([ord_C(k) / ORD for k in get_all_cycle_types(N)], 
                         dtype=np.float128))
print("Sigma:\n", Sigma, "\n")

# calculate Psi * Sigma * Psi.T
PSPT = np.around(Psi @ Sigma @ Psi.T).astype(np.uint64)
print("Psi * Sigma * Psi.T:\n", PSPT, "\n")

Psi:
 [[120   0   0   0   0   0   0]
 [ 60   6   0   0   0   0   0]
 [ 30   6   2   0   0   0   0]
 [ 20   6   0   2   0   0   0]
 [ 10   4   2   1   1   0   0]
 [  5   3   1   2   0   1   0]
 [  1   1   1   1   1   1   1]] 

Sigma:
 [[0.00833333 0.         0.         0.         0.         0.
  0.        ]
 [0.         0.08333333 0.         0.         0.         0.
  0.        ]
 [0.         0.         0.125      0.         0.         0.
  0.        ]
 [0.         0.         0.         0.16666667 0.         0.
  0.        ]
 [0.         0.         0.         0.         0.16666667 0.
  0.        ]
 [0.         0.         0.         0.         0.         0.25
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.2       ]] 

Psi * Sigma * Psi.T:
 [[120  60  30  20  10   5   1]
 [ 60  33  18  13   7   4   1]
 [ 30  18  11   8   5   3   1]
 [ 20  13   8   7   4   3   1]
 [ 10   7   5   4   3   2   1]
 [  5   4   3   3   2   2   1]
 [  1   1   1   1   1   1   1]] 



# How to determine K?

We can see that
$$ A_{ij} := [KK^T]_{ij} = \sum_{k=j}^n K_{ik}K_{jk} $$

The last column can be easily determined by calculating 
$$ K_{nn} = \sqrt{A_{nn}}, \quad K_{kn} = \frac{A_{kn}}{K_{nn}} $$

Then we can define a matrix
$$ [B]_{ij} := K_{in}K_{jn} $$

And construct the matrix
$$ \tilde{A} := A - B $$

Which is now effectively a lower dimensional version of of our initial problem, 
so we can just begin all over again and solve for K recursively!

In [5]:
# initialize K with zeros
K = np.zeros(shape=(len(PSPT), len(PSPT))).tolist()

def fill_K_column(A, n):
    if len(A) == 0:
        return
    # fill the n'th column of K, note that indices start at 0 here...
    K[n-1][n-1] = float(np.sqrt(A[n-1, n-1]))
    for i in range(n):
        K[i][n-1] = float(A[i, n-1]/K[n-1][n-1])
    
    # build B matrix
    K_arr = np.array(K)
    B = np.fromfunction(lambda i, j: K_arr[i, n-1]*K_arr[j, n-1], 
                         shape=(n, n), dtype="i4")
    
    # build A tilde and recurse
    A_tilde = (A - B)[:-1, :-1]
    return fill_K_column(A_tilde, n-1)

# start of recursion
fill_K_column(PSPT, len(PSPT))

K = np.around(np.array(K)).astype(np.uint64)
print("K:\n", K, "\n")

K:
 [[1 4 5 6 5 4 1]
 [0 1 2 3 3 3 1]
 [0 0 1 1 2 2 1]
 [0 0 0 1 1 2 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1]] 



In [6]:
X = np.around(np.linalg.inv(K) @ Psi).astype(np.int64)
print("X:\n", X, "\n")

II = np.around(X @ Sigma @ X.T).astype(np.int64)
success = not np.max(np.abs(II - np.identity(len(II))))
print("X * Sigma * X.T:\n", II, "\n")
print("Is X * Sigma * X.T == I (after rounding to integers)?:", success, "\n")

if success:
    print("We calculated the character table succesfully!")
    if SAVE:
            with open("character_table_of_S_"+str(N)+".npy", "wb") as f:
                np.save(f, X)
else:
    print("It seems like something went wrong :(")

X:
 [[ 1 -1  1  1 -1 -1  1]
 [ 4 -2  0  1  1  0 -1]
 [ 5 -1  1 -1 -1  1  0]
 [ 6  0 -2  0  0  0  1]
 [ 5  1  1 -1  1 -1  0]
 [ 4  2  0  1 -1  0 -1]
 [ 1  1  1  1  1  1  1]] 

X * Sigma * X.T:
 [[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]] 

Is X * Sigma * X.T == I (after rounding to integers)?: True 

We calculated the character table succesfully!
